# xmon1 <-> xmon2 <-> (xmon3, xmon4)

In [108]:
import numpy as np
import numpy.linalg as alg
import scqubits as scq
import sys
sys.path.insert(0, '..')
from QSwitch import QSwitch

In [109]:
hbar = 1.054e-34
h = hbar*2*np.pi
qe = 1.609e-19
flux_quant = h/2/qe
red_flux_quant = hbar/2/qe 

# Unit conversions
MHz = 10.0**(-3)
GHz = 1.0
kHz = 10.0**(-6)
us = 10.0**3
ns = 1.0

In [110]:
# Q3D VALUES
# All in SI units
Cjs = np.array([78.490, 136.330, 68.713, 87.379])*1e-15 # fF -> F
# Cjs = np.array([80.874, 140.519, 69.822, 89.644])*1e-15 # fF -> F
Cab = np.array([3.981, 3.858, 3.477])*1e-15 # [C12, C23, C24] fF -> F
# Cab = np.array([2.916, 2.868, 2.477])*1e-15 # [C12, C23, C24] fF -> F
# Cab = np.array([6.587, 6.479, 6.036])*1e-15 # [C12, C23, C24] fF -> F
Crqs = np.array([7.742, 14.431, 7.336, 7.769])*1e-15 # fF -> F
Cds = np.array([79, 118, 70, 66])*1e-18 # aF -> F

# HFSS VALUES
res_freqs = np.array([6.882, 5.643, 6.174, 6.470]) # GHz
# res_freqs = np.array([6, 4.5, 5.2, 5.4]) # GHz
# res_freqs = np.array([7.6, 5.7, 6.8, 6.9]) # GHz
Q_res = np.array([1.272e5, 2.860e5, 7.677e4, 1.526e5])
Q_drives = np.array([np.inf, 8.159e6, np.inf, np.inf]) # currently only simulated for qubit 2

# GET ZO FROM CPW MATHEMATICA
Zo = 52.7066 # Ohm

Ej_targets = np.array([14.9, 13.91, 9.93, 13.91])*1e9 # GHz -> Hz
# Ej_targets = np.array([10.53, 9.73, 6.95, 9.73])*1e9 # GHz -> Hz
# Ej_targets = np.array([19.5, 18.28, 12.91, 18.28])*1e9 # GHz -> Hz
Ljs = flux_quant**2/h/4/np.pi**2/Ej_targets # H

c_matrix = np.array([
    [Cab[0]+Cjs[0], -Cab[0], 0, 0],
    [-Cab[0], sum(Cab)+Cjs[1], -Cab[1], -Cab[2]],
    [0, -Cab[1], Cab[1]+Cjs[2], 0],
    [0, -Cab[2], 0, Cab[2]+Cjs[3]],
])

inv_l_matrix = np.diag(1/Ljs)

print('Check symmetric:', np.allclose(c_matrix, c_matrix.T), np.allclose(inv_l_matrix, inv_l_matrix.T))

Check symmetric: True True


# BEGIN ANALYSIS

In [111]:
inv_c_matrix = alg.inv(c_matrix)

evals_c, u1 = alg.eigh(inv_c_matrix)
u1 = np.array([u1[i]/np.sqrt(evals_c[i]) for i in range(len(evals_c))])
evals, evecs = alg.eigh(u1.transpose() @ inv_l_matrix @ u1)
print('Modes (cols are evecs)\n', np.around(evecs, 3))

# ======== IDK WHY BUT THIS DOESN'T WORK ========
# Look for col with max in Q1_NODE -> use q_modes to reference in EC, EJ
# q_modes = np.array([np.argmax(np.abs(evecs[i,:])) for i in range(4)])
# print('Qubit modes were mapped', f'q1: mode {q_modes[0]},', f'q2: mode {q_modes[1]},', f'q3: mode {q_modes[2]},', f'q4: mode {q_modes[3]}')
# old_evecs = evecs.copy()
# for i in range(4):
#     evecs[:,i] = old_evecs[:,q_modes[i]]
# # evals = np.array([evals[q_modes[i]] for i in range(4)])
# ECs = qe**2/2/h * evecs.transpose() @ inv_c_matrix @ evecs
# EJs = flux_quant**2/h/4/np.pi**2 * evecs.transpose() @ inv_l_matrix @ evecs
# ===============================================

ECs = qe**2/2/h * inv_c_matrix
EJs = flux_quant**2/h/4/np.pi**2 * inv_l_matrix
Zs = np.sqrt(8*np.diag(ECs)/np.diag(EJs))
gs = np.array([[4*ECs[i,j]/np.sqrt(Zs[i]*Zs[j]) for i in range(len(c_matrix))] for j in range(len(c_matrix))])

print('all ECs\n', 1e-9*ECs)
print('all EJs\n', 1e-9*ECs)
print('all q-q gs\n', 1e-6*gs)
print('impedances (unitless)\n', *Zs)

ECs = 1e-9 * np.diag(ECs)
EJs = 1e-9 * np.diag(EJs)

gs = 1e-9 * np.abs(np.array([gs[0,1], gs[1,2], gs[1,3]])) # g12, g23, g24

# AFTER HERE EVERYTHING IS INDEXED SO QUBIT NUMBER = INDEX AND EVERYTHING IS IN GHz


Modes (cols are evecs)
 [[-0.051  0.06   0.995 -0.06 ]
 [ 0.012  0.998 -0.058  0.028]
 [ 0.024  0.024 -0.061 -0.998]
 [ 0.998 -0.01   0.053  0.021]]
all ECs
 [[2.37316141e-01 6.41347778e-03 3.40951582e-04 2.45439621e-04]
 [6.41347778e-03 1.32862579e-01 7.06320471e-03 5.08456444e-03]
 [3.40951582e-04 7.06320471e-03 2.69714035e-01 2.70304248e-04]
 [2.45439621e-04 5.08456444e-03 2.70304248e-04 2.15328062e-01]]
all EJs
 [[2.37316141e-01 6.41347778e-03 3.40951582e-04 2.45439621e-04]
 [6.41347778e-03 1.32862579e-01 7.06320471e-03 5.08456444e-03]
 [3.40951582e-04 7.06320471e-03 2.69714035e-01 2.70304248e-04]
 [2.45439621e-04 5.08456444e-03 2.70304248e-04 2.15328062e-01]]
all q-q gs
 [[2659.32716843   81.66847636    3.34336569    2.77000812]
 [  81.66847636 1922.5599967    78.70626979   65.20884245]
 [   3.34336569   78.70626979 2314.4158508     2.66953684]
 [   2.77000812   65.20884245    2.66953684 2447.53481807]]
impedances (unitless)
 0.3569566669027231 0.27642846825269585 0.46614619351470

In [112]:
cutoffs = [4, 5, 4, 4]
isCavity = [False, False, False, False]

qram = QSwitch(
    EJs=EJs,
    ECs=ECs,
    gs=gs,
    cutoffs=cutoffs,
    isCavity=isCavity,
)

qubit_freqs = np.array(qram.qubit_freqs)
alphas = qram.alphas
charge_mat_el = 1/np.sqrt(2) * (EJs/8/ECs)**1/4
g_rqs = Crqs/(Crqs+Cjs) * res_freqs * np.sqrt(2*Zo/(hbar/qe**2))
deltas = qubit_freqs-res_freqs
lamb_shifts = g_rqs**2/deltas
chi_shifts = g_rqs**2/deltas * alphas/(deltas+alphas)
drive_purcell_rate_analytical = (2*np.pi*qubit_freqs*1e9*Cds)**2/(Cjs+Cds) * 50 /2/np.pi
drive_purcell_rate_sim = qubit_freqs*1e9/Q_drives
kappas = res_freqs/Q_res
res_purcell_rate = kappas * (g_rqs/deltas)**2

print('EC values (GHz):\n\t', *ECs)
print('EJ values (GHz):\n\t', *EJs)
print('qubit freqs/2pi (GHz):\n\t', *qubit_freqs)
print('alphas/2pi (GHz):\n\t', *alphas)
print('q-q couplings g12, g23, g24 (MHz)\n\t', *(gs*1e3))
print('q-res couplings (MHz) [want 50-100]:\n\t', *(1e3*g_rqs))
print('delta wq-wr (GHz) [want 1.5-2]:\n\t', *deltas)
print('lamb shifts (MHz):\n\t', *(1e3*lamb_shifts))
print('chi shifts (MHz) [want 0.15-0.5]:\n\t', *(1e3*chi_shifts))
print('\n')
print('drive purcell limits - analytical (ms) [want >1 ms]:\n\t', *(1e3/drive_purcell_rate_analytical))
print('drive purcell limits - simulated (ms) [want >1 ms]:\n\t', *(1e3/drive_purcell_rate_sim))
print('res linewidth kappas - simulated (kHz):\n\t', *(1e6*kappas))
print('res purcell limits - simulated (s):\n\t', *(1/res_purcell_rate))

EC values (GHz):
	 0.23731614056123682 0.13286257875274943 0.2697140347651836 0.2153280620291862
EJ values (GHz):
	 14.9 13.91 9.930000000000001 13.91
qubit freqs/2pi (GHz):
	 5.069258499685212 3.7071885004802088 4.3403653553659405 4.668958740871519
alphas/2pi (GHz):
	 -0.2676605209663876 -0.1452030948815537 -0.3210446073938158 -0.2423543381126212
q-q couplings g12, g23, g24 (MHz)
	 81.66847636073615 78.70626978513529 65.20884244800864
q-res couplings (MHz) [want 50-100]:
	 99.42202710084078 86.91619483782115 95.83313100669713 85.00666448105575
delta wq-wr (GHz) [want 1.5-2]:
	 -1.8127415003147878 -1.935811499519791 -1.83363464463406 -1.801041259128481
lamb shifts (MHz):
	 -5.452922808422383 -3.9024589568561208 -5.008625368975635 -4.0121973716979085
chi shifts (MHz) [want 0.15-0.5]:
	 -0.7015625560645282 -0.27229463921504066 -0.7462791334961466 -0.4758615706659536


drive purcell limits - analytical (ms) [want >1 ms]:
	 1.5594027447691334 2.2696764342614992 2.3718246839177026 2.9312790

 /Users/conniemiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py: 35